In [10]:
import pandas as pd
file_path = './travel_dataset_cleaned_ciudad.csv'
data = pd.read_csv(file_path)
data.head()

,Nombre,Calificación,Precio,lat,lng,Relajación,Aventura,Cultural,Histórico,Espiritual,...,ciudad_Washington,ciudad_West End,ciudad_Westmoreland Parish,ciudad_Weston,ciudad_Yamanashi,ciudad_Yucatán,ciudad_Zacapa,ciudad_Zacapa Department,ciudad_Zulia,ciudad_Zurich
0,Times Square,4.7,600.0,40.757975,-73.985543,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Henry Hudson Bridge,4.2,600.0,40.877713,-73.922431,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Lincoln Center for the Performing Arts,4.8,600.0,40.772464,-73.983489,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Radio City Music Hall,4.8,600.0,40.759976,-73.979977,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Rockefeller Center,4.7,600.0,40.758740,-73.978674,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:

# Tomar muestra de 10000 registros
df_sample = data.sample(n=5022, random_state=42)

# Convertir a JSON
json_data = df_sample.to_json(orient='records', force_ascii=False, indent=2)

# Guardar en un archivo
with open('data.json', 'w', encoding='utf-8') as f:
    f.write(json_data)

In [14]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Eliminar las columnas 'Nombre'
data_cleaned = data.drop(columns=['Nombre'])

# Normalizar los datos
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_cleaned)

# Entrenar el modelo K-means con 8 clústeres
kmeans = KMeans(n_clusters=8, random_state=42)
kmeans.fit(data_scaled)

# Agregar la etiqueta de los clústeres al dataset original
data['Cluster'] = kmeans.labels_

# Mostrar los primeros resultados
data[['Nombre', 'Cluster']].head()

,Nombre,Cluster
0,Times Square,2
1,Henry Hudson Bridge,2
2,Lincoln Center for the Performing Arts,2
3,Radio City Music Hall,2
4,Rockefeller Center,2


In [15]:
data

,Nombre,Calificación,Precio,lat,lng,Relajación,Aventura,Cultural,Histórico,Espiritual,...,ciudad_West End,ciudad_Westmoreland Parish,ciudad_Weston,ciudad_Yamanashi,ciudad_Yucatán,ciudad_Zacapa,ciudad_Zacapa Department,ciudad_Zulia,ciudad_Zurich,Cluster
0,Times Square,4.7,600.0,40.757975,-73.985543,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2
1,Henry Hudson Bridge,4.2,600.0,40.877713,-73.922431,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2
2,Lincoln Center for the Performing Arts,4.8,600.0,40.772464,-73.983489,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2
3,Radio City Music Hall,4.8,600.0,40.759976,-73.979977,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2
4,Rockefeller Center,4.7,600.0,40.758740,-73.978674,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5017,Melbourne River Cruises,4.4,150.0,-37.819918,144.965806,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2
5018,Kings Park,4.3,150.0,-37.895767,145.301217,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2
5019,Funfields Themepark,4.4,150.0,-37.519994,145.111246,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,2
5020,Diamond Valley Railway Inc.,4.8,150.0,-37.733599,145.141934,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2


In [16]:
import pandas as pd
import numpy as np

def recomendar_lugares(region, pais, ciudad, tipo_turismo, presupuesto, dataset, kmeans_model):
    # Filtrar por región y país
    region_column = f"region_{region}"
    pais_column = f"pais_{pais}"
    ciudad_column = f"ciudad_{ciudad}"


    if region_column not in dataset.columns or pais_column not in dataset.columns or ciudad_column not in dataset.columns:
        return f"La región '{region}', el país '{pais}' o la ciudad '{ciudad}' no están en el dataset."

    filtro_region_pais = dataset[(dataset[region_column] == 1) & (dataset[pais_column] == 1)]

    filtro_region_pais_ciudad = filtro_region_pais[filtro_region_pais[ciudad_column] == 1]

    # Filtrar por tipo de turismo
    filtro_turismo = filtro_region_pais_ciudad[filtro_region_pais_ciudad[tipo_turismo] == 1]

    # Filtrar por presupuesto
    filtro_presupuesto = filtro_turismo[filtro_turismo['Precio'] <= presupuesto]

    # Si no hay resultados después de filtrar
    if filtro_presupuesto.empty:
        return "No se encontraron lugares que coincidan con los criterios seleccionados."

    # Preprocesar los datos para el modelo K-means
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(filtro_presupuesto.drop(columns=['Nombre','Cluster']))

    # Predecir el clúster para cada lugar filtrado
    clusters = kmeans_model.predict(data_scaled)
    filtro_presupuesto['PredictedCluster'] = clusters

    # Seleccionar lugares aleatoriamente de los clústeres
    cluster_indices = np.unique(clusters)
    recomendados = pd.DataFrame()

    for cluster_index in cluster_indices:
        cluster_lugares = filtro_presupuesto[filtro_presupuesto['PredictedCluster'] == cluster_index]
        if not cluster_lugares.empty:
            num_samples = min(3, len(cluster_lugares))  # Seleccionar como máximo 2 lugares, pero no más de los disponibles
            recomendados = pd.concat([recomendados, cluster_lugares.sample(n=num_samples)])  # Agregar un lugar aleatorio por clúster

    # Seleccionar un máximo de 3 lugares aleatorios de los recomendados
    if len(recomendados) > 3:
        recomendados = recomendados.head(3)

    return recomendados[['Nombre', tipo_turismo, 'Calificación','Precio']]

region = "Centroamerica"
pais = "Honduras"
ciudad = "Departamento de Copán"
tipo_turismo = "Cultural"
presupuesto = 1000
dataset = data  # Usando el dataset original con los clusters

recomendaciones = recomendar_lugares(region, pais, ciudad, tipo_turismo, presupuesto, dataset, kmeans)
recomendaciones

,Nombre,Cultural,Calificación,Precio
1698,La Pintada Tours,1,5.0,150.0
1660,Copán Ruins,1,4.8,150.0
1649,Museo de esculturas,1,4.6,150.0


In [6]:
import joblib
import numpy as np

# Suponiendo que ya tienes el modelo de K-Means entrenado con `kmeans`
centroids = kmeans.cluster_centers_

# Guardar los centroides en un archivo
joblib.dump(centroids, 'kmeans_centroids.pkl')

['kmeans_centroids.pkl']

In [7]:
import json
import joblib

# Cargar los centroides guardados
centroids = joblib.load('kmeans_centroids.pkl')

# Convertir los centroides a una lista de listas
centroids_list = centroids.tolist()

# Guardar los centroides en un archivo JSON
with open('centroids.json', 'w') as f:
    json.dump(centroids_list, f)